In [1]:
#Importing necessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl 
import json

#### Create list of all committee ids for use in future functions 

In [2]:
# Creating a list of all id numbers for future funtion
list_data=pd.read_excel(r'C:\Users\bboul\Downloads\committees.xlsx') 
id_list=[i for i in list_data['committee_id']]
print(id_list)

['C00360354', 'C30002430', 'C00252338', 'C30001788', 'C00106146', 'C30001309', 'C00000422', 'C00190272', 'C00678722', 'C00008839', 'C00114132', 'C00651265', 'C00152660', 'C00301358', 'C00197202', 'C00220608', 'C00276311', 'C00312223', 'C00236323', 'C00457242', 'C00084061', 'C00406850', 'C00135202', 'C00194746', 'C00199166', 'C00215202', 'C00523217', 'C00211375', 'C00582114', 'C00248716', 'C00303024', 'C00502906', 'C00436931', 'C00488742', 'C00030718', 'C00010082', 'C00082040', 'C30001101']


#### Read Data

In [3]:
# Creating an empty list to append the dataframes. Then I use pd.concat() to bring all these dataframes together. 
data_frames=[]
for num in id_list:
    num = pd.read_csv(f'C:/Users/bboul/OneDrive/Documents/fec_distributions_data/disbursements_{num}.csv', low_memory=False)
    data_frames.append(num)
data=pd.concat(data_frames)

# Delete columns that consist of 100 percent null values.
data.dropna(axis=1, how='all', inplace=True)

#Printing Duplicate and columns and rows in dataset.
print('Duplicated Row: ' +str(data.duplicated().sum()))
print('Columns: ' + str(len(data.columns)), 'Rows: ' + str(len(data.index)))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Null Values: \n' + str(data.isna().sum()))   


Duplicated Row: 0
Columns: 69 Rows: 200451
Null Values: 
disbursement_type_description     31946
file_number                       65299
disbursement_description          44857
two_year_transaction_period           0
filing_form                           0
candidate_office                  50692
comm_dt                          200184
candidate_name                    47588
schedule_type                         0
spender_committee_type                0
image_number                          0
schedule_type_full                  297
link_id                               0
line_number                       69701
election_type                     23943
payee_prefix                     200246
payee_middle_name                200173
amendment_indicator_desc              2
disbursement_date                    24
category_code                    106704
committee_id                          0
report_year                           0
semi_annual_bundled_refund       109137
spender_committee_desig

##### Removind nested dictionary from api data


In [4]:
# Retreiving row where recipient committee not null.
recip_com_column_data=data.loc[data['recipient_committee'].notnull()]
# Retrieve key, values from nested dictionary
dict_keys=recip_com_column_data['recipient_committee'].apply(eval).iat[0].keys()
# Creating nested dictionary using keys and '0' 
empty_dict=dict(zip(dict_keys, ['0']*len(dict_keys)))
# Fill NaN rows with empty dictionary 
data.loc[:,'recipient_committee'].fillna(str(empty_dict), inplace=True)


# Reading json format inside column cell and creating table
recipient_data = pd.json_normalize(data['recipient_committee'].apply(eval))

# Loop to add committee_ to column name for clarity of whole dataset
for column in recipient_data.columns:
    if 'recipient_committee' in column:
        pass
    else:
        recipient_data.rename(columns={f'{column}':'recipient_committee_'+str(column)}, inplace=True)

#Dropping columns that consistent of more than 50% null values 
threshold = len(recipient_data.index)/2
recipient_data.dropna(axis= 1, thresh= threshold, inplace=True)
data.drop(columns=['recipient_committee'], inplace=True)
recipient_data.drop(columns=['recipient_committee_state', 'recipient_committee_zip'])

#Showing columns and rows of our dataset, also the null values in each column after cleaning.
print('Columns: ' + str(len(recipient_data.columns)), 'Rows: ' + str(len(recipient_data.index)))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Null Values' +'\n'+ str(recipient_data.isna().sum()))

Columns: 30 Rows: 200451
Null Values
recipient_committee_last_file_date                   3
recipient_committee_committee_type                   0
recipient_committee_name                             0
recipient_committee_last_cycle_has_activity          3
recipient_committee_filing_frequency                 0
recipient_committee_street_1                         2
recipient_committee_zip                             27
recipient_committee_cycles_has_financial            30
recipient_committee_first_file_date                  3
recipient_committee_designation_full                 0
recipient_committee_first_f1_date                   76
recipient_committee_treasurer_name                  64
recipient_committee_committee_type_full              0
recipient_committee_convert_to_pac_flag              0
recipient_committee_is_active                        0
recipient_committee_committee_id                     0
recipient_committee_state                           31
recipient_committee_city    

##### Removind nested dictionary from api data


In [5]:

# reading json format inside column cell and creating table
committee = pd.json_normalize(data['committee'].apply(eval))

# Loop to add committee_ to column name for readablity of table
for column in committee.columns:
    if 'committee' in column:
        pass
    else:
        committee.rename(columns={f'{column}':'committee_'+str(column)}, inplace=True)

#Dropping columns that consist of all null values 
committee.dropna(axis=1, how='all', inplace=True)
data.drop(columns=['committee','committee_id'], inplace=True)

#Showing columns and rows of our dataset, also the null values in each column after cleaning.
print('Columns: ' + str(len(committee.columns)), 'Rows: ' + str(len(committee.index)))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Null Values: \n' + str(committee.isna().sum()))

Columns: 33 Rows: 200451
Null Values: 
committee_last_file_date                   0
committee_type                             0
committee_name                             0
committee_last_cycle_has_activity          0
committee_filing_frequency                 0
committee_street_1                         0
committee_zip                              0
committee_cycles_has_financial             0
committee_first_file_date                  0
committee_designation_full                 0
committee_first_f1_date                  268
committee_treasurer_name                 268
committee_type_full                        0
committee_convert_to_pac_flag              0
committee_is_active                        0
committee_id                               0
committee_state                            0
committee_city                             0
committee_designation                      0
affiliated_committee_name               1912
committee_street_2                    137702
committee_state_

##### Joining all tables and removing columns that consist of 100% Null Values

In [6]:
# Reset index to concat tables
data = pd.concat([data.reset_index(drop=True), committee, recipient_data], axis=1)

# Dropping 100% null columns
data.dropna(axis=1, how='all', inplace= True)

#### Fill in other NaN values with '0' for undesignated

In [7]:
data.fillna('0',inplace= True)

##### Delete any disbursements that could not be attributed to a recipient.

In [8]:
# retrieving the rows that are NaN in recipient name and recipient committee
# assigning values before they change in code.
total_disburment= int(data['disbursement_amount'].sum())
total_values=len(data.index)
no_recipient=data.loc[(data['recipient_name']== '0')&(data['recipient_committee_name']== '0')]
data.drop(no_recipient.index, inplace=True)

# printing rows removed and dollar amount lost.
print('Rows Removed: '+ str(len(no_recipient.index))+' of ' +str(total_values))
print('Dollar amount removed: $' + str(int(no_recipient['disbursement_amount'].sum()))+ ' out of $' + str(total_disburment))

Rows Removed: 1789 of 200451
Dollar amount removed: $2209000 out of $420395613


##### Blending recipient name and recipient committee name, also clearing discrepancies in recipient name.

In [9]:
# This code blends both columns recipient name and recipient committee name into one column
recipient_name_empty= data['recipient_name']=='0'
data=data.assign(recipient_parent_name= np.where(recipient_name_empty, data.recipient_committee_name, data.recipient_name))
data['recipient_parent_name'].value_counts()

HOYER FOR CONGRESS                                    587
UPTON FOR ALL OF US                                   465
FLORIDA RPAC                                          435
PALLONE FOR CONGRESS                                  418
DEMOCRATIC SENATORIAL CAMPAIGN COMMITTEE              408
                                                     ... 
FREDERICK POLLS, LLC                                    1
CLIFFORD FOR CONGRESS                                   1
KEEPING REPUBLICAN IDEAS STRONG TIMELY & INVENTIVE      1
SPELLMAN, HANK MATTHEW MD                               1
FRIENDS OF BILL PEACOCK                                 1
Name: recipient_parent_name, Length: 13905, dtype: int64

##### Creating a parent committee name using committee id from committee.xlsx to avoid ambiguity in committee_name

In [10]:
# Uses data we used earlier creating a groupby object and using the first entry. 
name_dict= list_data.groupby('committee_id')['name'].first()
# Creating a dictionary from the preivous data
name_dict= name_dict.to_dict()

# Mapping this information to a new column using committee id. 
data['parent_name'] = data['committee_id'].map(name_dict)
data['parent_name'].value_counts()

NATIONAL ASSOCIATION OF REALTORS    56595
LOCKHEED MARTIN                     22236
AMERICAN MEDICAL ASSOCIATION        21667
AMERICAN HOSPITAL ASSOCIATION       21067
BLUE CROSS BLUE SHIELD              20713
COMCAST                             18179
AMERICAN MEDICAL ASSOCIATION        14801
NCTA                                 9508
US CHAMBER OF COMMERCE               5051
AMAZON                               3686
AMERICAN CHEMISTRY COUNCIL           3468
META                                 1635
BUSINESS ROUNDTABLE                    56
Name: parent_name, dtype: int64

##### Identifying the way disbursements are catgeorized.

In [11]:
data['disbursement_purpose_category'].value_counts()

CONTRIBUTIONS      160914
OTHER               35049
REFUNDS              1038
TRANSFERS             911
MATERIALS             469
ADVERTISING           172
ADMINISTRATIVE         53
FUNDRAISING            51
LOAN-REPAYMENTS         3
TRAVEL                  2
Name: disbursement_purpose_category, dtype: int64

##### Investigating the 'OTHER' category further and removing data that can not be categorized by how disbursement amount was utilized

In [12]:
# Grouping the dataframe by disbursement_purpose_category 
other_category= data.groupby('disbursement_purpose_category')

# Selecting the category of 'OTHER' to further investigate the data.
other_category=other_category.get_group('OTHER')

# Remove data that can not be identified under a category other than 'OTHER'
no_description= other_category.loc[(other_category['disbursement_type_description']== '0')&(other_category['disbursement_description']== '0')]
data.drop(no_description.index, inplace=True)

# printing rows removed and dollar amount lost.
print('Rows Removed: '+ str(len(no_description.index))+ ' out of $' + str(total_values))
print('Dollar amount removed: $' + str(int(no_description['disbursement_amount'].sum()))+ ' out of $' + str(total_disburment))

Rows Removed: 957 out of $200451
Dollar amount removed: $3210865 out of $420395613


In [13]:
# Grouping the dataframe by disbursement_purpose_category 
other_category= data.groupby('disbursement_purpose_category')

# Selecting the category of 'OTHER' to further investigate the data.
other_category=other_category.get_group('OTHER')

kind= other_category.loc[other_category['disbursement_description'].str.contains('IN-KIND CONTRIB')]

In [14]:
other_category['disbursement_description'].value_counts().nlargest(20)

REFUND PER CO-OP AGREEMENT                                   6878
IN-KIND CONTRIB MADE TO REG. FILER                           3849
NONFEDERAL CONTRIBUTION                                      3648
DI-DIRECT CONTRIBUTION                                       1717
CONTRIBUTION                                                 1427
BANK FEE                                                      513
MERCHANT FEES                                                 485
TRANSFER OF FUNDS JOINT FUNDRAISING                           403
70% PER CO-OP AGREEMENT                                       373
BANK FEES                                                     372
MERCHANT FEE                                                  306
70% PER CO-OPERATIVE AGREEMENT                                271
CREDIT CARD BANK CHARGES                                      241
MERCHANT CREDIT CARD FEES                                     215
CONTRIBUTION TO A NONFEDERAL CAMPAIGN                         172
BANK FEE C

In [15]:
kind['disbursement_type_description'].value_counts()

IN-KIND CONTRIBUTION MADE TO REGISTERED FILER    3884
Name: disbursement_type_description, dtype: int64

##### Identifying contributions in 'OTHER' further

In [16]:
# Dictionary of key words used in disbursement_description and values of proper purpose category.
filter_other= {'REFUND':'REFUNDS', 'PER CO':'REFUNDS', 'CONTRIB':'CONTRIBUTIONS', 'IN-KIND':'IN-KIND: CONTRIBUTIONS', 'ELECTIONEERING':'ADVERTISING', 'TRANS':'TRANSFERS', 'TRSF':'TRANSFERS'}

# For loop to check for key in disbursement_description:
for key,value in filter_other.items():
    # Creates condtion to use inside pn.where()
    contrib_in_other= (data['disbursement_purpose_category']=='OTHER')&(data['disbursement_description'].str.contains(f'{key}'))

    # Using pn.where to recategorize the row prope.
    data['disbursement_purpose_category'] = np.where(contrib_in_other, f'{value}', data['disbursement_purpose_category'])

##### Purpose Category after cleaning other category

In [17]:
data['disbursement_purpose_category'].value_counts()

CONTRIBUTIONS             173567
OTHER                      11696
REFUNDS                     9580
TRANSFERS                   1753
MATERIALS                    469
IN-KIND: CONTRIBUTIONS       359
ADVERTISING                  172
ADMINISTRATIVE                53
FUNDRAISING                   51
LOAN-REPAYMENTS                3
TRAVEL                         2
Name: disbursement_purpose_category, dtype: int64

#####  Verifying refund is all negative values

In [18]:
refunds = data.loc[data['disbursement_purpose_category'] == 'REFUNDS']
refunds['disbursement_amount'].apply(lambda x: x*-1 if x == 0 else x)

841        199.92
1847      2400.00
1897      1200.00
1898       420.00
1899      2600.00
           ...   
194950     500.00
195435    5000.00
195562     500.00
196426    5000.00
197222     700.00
Name: disbursement_amount, Length: 9580, dtype: float64

##### Reviewing final dataset shape and null values in columns

In [19]:
#Showing final count of dataset with all the null values in the columns.
print('Columns: ' + str(len(data.columns)), 'Rows: ' + str(len(data.index)))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print('Null Values: ' + str(data.isnull().sum()))

Columns: 131 Rows: 197705
Null Values: disbursement_type_description                    0
file_number                                      0
disbursement_description                         0
two_year_transaction_period                      0
filing_form                                      0
candidate_office                                 0
comm_dt                                          0
candidate_name                                   0
schedule_type                                    0
spender_committee_type                           0
image_number                                     0
schedule_type_full                               0
link_id                                          0
line_number                                      0
election_type                                    0
payee_prefix                                     0
payee_middle_name                                0
amendment_indicator_desc                         0
disbursement_date                          

In [20]:
data.head(10)

,disbursement_type_description,file_number,disbursement_description,two_year_transaction_period,filing_form,candidate_office,comm_dt,candidate_name,schedule_type,spender_committee_type,...,recipient_committee_cycles_has_activity,recipient_committee_party_full,recipient_committee_last_cycle_has_financial,recipient_committee_party,recipient_committee_cycles,recipient_committee_last_f1_date,recipient_committee_cycle,recipient_committee_jfc_committee,recipient_parent_name,parent_name
0,0,1603117.0,MERCHANT FEES,2022,F3X,0,0,0,SB,Q,...,0,0,0,0,0,0,0,0,"AMAZON PAYMENTS, INC.",AMAZON
1,CONTRIBUTION MADE TO NON-AFFILIATED,1603117.0,2022 GENERAL,2022,F3X,H,0,"VEASEY, MARC A.",SB,Q,...,"[2012, 2014, 2016, 2018, 2020, 2022]",DEMOCRATIC PARTY,2022,DEM,"[2012, 2014, 2016, 2018, 2020, 2022]",2014-09-04,2022,"[{'joint_committee_name': 'MARC PAC', 'joint_c...",MARC VEASEY CONGRESSIONAL CAMPAIGN COMMITTEE,AMAZON
2,0,1603117.0,BANK FEES,2022,F3X,0,0,0,SB,Q,...,0,0,0,0,0,0,0,0,COMERICA BANK,AMAZON
3,CONTRIBUTION MADE TO NON-AFFILIATED,1603117.0,2022 CONTRIBUTION,2022,F3X,0,0,0,SB,Q,...,"[2002, 2004, 2006, 2008, 2010, 2012, 2014, 201...",REPUBLICAN PARTY,2022,REP,"[2002, 2004, 2006, 2008, 2010, 2012, 2014, 201...",2022-04-15,2022,"[{'joint_committee_name': None, 'joint_committ...",RESPONSIBILITY AND FREEDOM WORK PAC (RFWPAC),AMAZON
4,0,1603117.0,BANK FEES,2022,F3X,0,0,0,SB,Q,...,0,0,0,0,0,0,0,0,COMERICA BANK,AMAZON
5,CONTRIBUTION MADE TO NON-AFFILIATED,1603117.0,2022 CONTRIBUTION,2022,F3X,0,0,0,SB,Q,...,"[2016, 2018, 2020, 2022]",0,2022,0,"[2016, 2018, 2020, 2022]",2016-12-15,2022,"[{'joint_committee_name': None, 'joint_committ...",SENSIBLE AMERICAN SOLUTIONS SUPPORTING EVERYON...,AMAZON
6,CONTRIBUTION MADE TO NON-AFFILIATED,1603117.0,2022 PRIMARY,2022,F3X,H,0,"MOOLENAAR, JOHN",SB,Q,...,"[2014, 2016, 2018, 2020, 2022]",REPUBLICAN PARTY,2022,REP,"[2014, 2016, 2018, 2020, 2022]",2022-02-09,2022,"[{'joint_committee_name': None, 'joint_committ...",MOOLENAAR FOR CONGRESS,AMAZON
7,CONTRIBUTION MADE TO NON-AFFILIATED,1603117.0,2022 GENERAL,2022,F3X,S,0,"BOOZMAN, JOHN NICHOLS",SB,Q,...,"[2010, 2012, 2014, 2016, 2018, 2020, 2022]",REPUBLICAN PARTY,2022,REP,"[2010, 2012, 2014, 2016, 2018, 2020, 2022]",2022-01-21,2022,"[{'joint_committee_name': None, 'joint_committ...",BOOZMAN FOR ARKANSAS,AMAZON
8,0,1596426.0,MERCHANT FEES,2022,F3X,0,0,0,SB,Q,...,0,0,0,0,0,0,0,0,"AMAZON PAYMENTS, INC.",AMAZON
9,CONTRIBUTION MADE TO NON-AFFILIATED,1596426.0,2022 CONTRIBUTION,2022,F3X,0,0,0,SB,Q,...,"[2014, 2016, 2018, 2020, 2022]",0,2022,0,"[2014, 2016, 2018, 2020, 2022]",2019-03-21,2022,"[{'joint_committee_name': None, 'joint_committ...",TRUE NORTH PAC,AMAZON


#Creating CSV to use in tableau for further analysis

In [ ]:
data.to_csv(r'C:/Users/bboul/OneDrive/Documents/fec_distributions_data/disbursements_master.csv', index=False)